<a href="https://colab.research.google.com/github/haizzzi/simple-nmt-simplied/blob/main/pre-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

구글 드라이브 마운트 위치 "/content/drive/MyDrive/"

In [6]:
!cat /content/drive/MyDrive/nmt_fold/data1/*.txt > combined.tsv

In [24]:
%cd /content/drive/MyDrive/nmt_fold/

/content/drive/MyDrive/nmt_fold


In [44]:
!iconv -f UTF-16LE -t UTF-8 /content/drive/MyDrive/nmt_fold/data1/combined.tsv > /content/drive/MyDrive/nmt_fold/data1/combined_utf8.tsv
!mv /content/drive/MyDrive/nmt_fold/data1/combined_utf8.tsv /content/drive/MyDrive/nmt_fold/data1/combined.tsv

In [49]:
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.tsv
!wc -l /content/drive/MyDrive/nmt_fold/data1/combined.tsv

﻿'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 앱입니다.	Bible Coloring' is a coloring application that allows you to experience beautiful stories in the Bible.
씨티은행에서 일하세요?	Do you work at a City bank?
푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.	"PURITO's bestseller, which recorded 4th rough -cuts by words of mouth from abroad."
11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.	In Chapter 11 Jesus called Lazarus from the tomb and raised him from the dead.
"6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다."	"I would feel grateful to know how many stocks will be secured of size 6.5, 7, and 8."
1602418 /content/drive/MyDrive/nmt_fold/data1/combined.tsv


In [48]:
!ls -l /content/drive/MyDrive/nmt_fold/data1/

total 1090100
-rw------- 1 root root  35448312 Aug 13 05:39 01.txt
-rw------- 1 root root  36863298 Aug 13 05:45 02.txt
-rw------- 1 root root  20178746 Aug 13 05:46 03.txt
-rw------- 1 root root 107751918 Aug 13 05:48 04.txt
-rw------- 1 root root  83395820 Aug 13 05:49 05.txt
-rw------- 1 root root 113856170 Aug 13 05:50 06.txt
-rw------- 1 root root  92307742 Aug 13 05:51 07.txt
-rw------- 1 root root  47288970 Aug 13 05:52 08.txt
-rw------- 1 root root  60739970 Aug 13 05:53 09.txt
-rw------- 1 root root  56909742 Aug 13 05:54 10.txt
-rw------- 1 root root 461519736 Aug 13 06:31 combined.tsv
